In [ ]:
!pip install --quiet  datasets #to access race dataset
!pip install --quiet pyarrow   #to deal with parquet files for saving dataset if required
!pip install --quiet  tqdm     #for progress bars
!pip install --quiet transformers # for t5 model
!pip install --quiet tokenizers  #tokenizers from HuggingFace
!pip install --quiet sentencepiece #subword tokenizer used by T5
!pip install --quiet pytorch-lightning # pytorch wrapper
!pip install --quiet torchtext # text utilities

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.8/722.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 59.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Fetching Datasets

In [ ]:
#imports
import pandas as pd
import torch
from tqdm import tqdm
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from pprint import pprint
import copy

Fetching Custom Dataset

In [ ]:
from google.colab import files
import io
import pandas as pd
uploaded = files.upload()
df2 = pd.read_csv(io.BytesIO(uploaded['Dataset - Sheet1.csv']))

KeyboardInterrupt: ignored

In [ ]:
df3=df2.iloc[0:177, 1:4]
df3

In [ ]:
def create_dataset(filename,  verbose = False):
  data  = pd.read_csv(filename, skipinitialspace=True, usecols=['Context','Question','Answer'] )
  result_df1  = pd.DataFrame(columns = ['context', 'question','answer'])
  result_df1 = data
  return result_df1

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df2, test_size=0.2)

In [ ]:
df_train1 , df_validation1 = create_pandas_dataset1(train) , create_pandas_dataset1(valid)
print(f"\n Total Train Samples:{df_train1.shape} , Total Validation Samples:{df_validation1.shape}")

In [ ]:
device  = 'cuda' if torch.cuda.is_available() else "cpu"

In [ ]:
def create_pandas_dataset1(data,
                          answer_threshold=512,
                          verbose = False):

  ''' Create a Pandas Dataframe from hugging face dataset.
  Params:
        answer_threshold: Only consider those Question Answer pairs where the Answer is short.
  '''
  #print(data)
  count_long ,count_short = 0 , 0
  result_df  = pd.DataFrame(columns = ['context', 'question','answer'])
  for val in enumerate(tqdm(data)):
      passage = val['Context']
      question = val['Question']
      answer = val['Answer']
      #answer = val['options'][ord(ans)-65]
      no_of_words = len(answer.split())
      if no_of_words >= answer_threshold:
          count_long = count_long + 1
          continue
      else:
          result_df.loc[count_short] = [passage] +[question] + [answer]
          count_short = count_short + 1
  if verbose:
    return (result_df,
            count_long,
            count_short)
  else:
    return result_df

In [ ]:
pd.options.display.max_rows , pd.options.display.max_columns  = 100,100

In [ ]:
def create_pandas_dataset(data,
                          answer_threshold=512,
                          verbose = False):

  ''' Create a Pandas Dataframe from hugging face dataset.
  Params:
        answer_threshold: Only consider those Question Answer pairs where the Answer is short.
  '''
  count_long ,count_short = 0 , 0
  result_df  = pd.DataFrame(columns = ['context', 'question','answer'])
  for index,val in enumerate(tqdm(data)):
      passage = val['article']
      question = val['question']
      ans = val['answer']
      answer = val['options'][ord(ans)-65]
      no_of_words = len(answer.split())
      if no_of_words >= answer_threshold:
          count_long = count_long + 1
          continue
      else:
          result_df.loc[count_short] = [passage] +[question] + [answer]
          count_short = count_short + 1
  if verbose:
    return (result_df,
            count_long,
            count_short)
  else:
    return result_df

#Changed it to Race Dataset

In [ ]:
train_dataset = load_dataset('race','middle', split='train')
valid_dataset = load_dataset('race', 'middle', split='validation')
print(f"Total Train Samples:{len(train_dataset)} , Total Validation Samples:{len(valid_dataset)}")

In [ ]:
print(train_dataset)

In [ ]:
df_train , df_validation = create_pandas_dataset(train_dataset) , create_pandas_dataset(valid_dataset)
print(f"\n Total Train Samples:{df_train.shape} , Total Validation Samples:{df_validation.shape}")

In [ ]:
# Saving data for future use
df_train.to_parquet('train_race.parquet')
df_validation.to_parquet('validation_race.parquet')

Run this one

In [ ]:
import pandas as pd
pd.read_parquet('/content/drive/MyDrive/train_race.parquet', engine='pyarrow')

# Creating a Pytorch DataSet for T5 Training and Validation

In [ ]:
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

In [ ]:
t5_tokenizer = T5Tokenizer.from_pretrained('t5-small',model_max_length=512)
t5_model = T5ForConditionalGeneration.from_pretrained('t5-small')

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [ ]:
class QuestionGenerationDataset(Dataset):
    def __init__(self, tokenizer, filepath, max_len_inp=512,max_len_out=512):
        self.path = filepath

        self.passage_column = "context"
        self.answer = "answer"
        self.question = "question"

        # self.data = pd.read_csv(self.path)
        self.data = pd.read_parquet(self.path).iloc[:2000,:]
        self.max_len_input = max_len_inp
        self.max_len_output = max_len_out
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []
        self._build()

    def __len__(self):
        return len(self.inputs)


    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask = self.inputs[index]["attention_mask"].squeeze()  #squeeze to get rid of the batch dimension
        target_mask = self.targets[index]["attention_mask"].squeeze()  # convert [batch,dim] to [dim]


        labels = copy.deepcopy(target_ids)
        labels [labels==0] = -100

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask,"labels":labels}

    def _build(self):
        for rownum,val in tqdm(self.data.iterrows()): # Iterating over the dataframe
            passage,answer,target = val[self.passage_column],val[self.answer],val[self.question]

            input_ = f"context: {passage}" # T5 Input format for question answering tasks
            target = f"question: {str(target)} answer: {answer}" # Output format we require
            # tokenize inputs
            tokenized_inputs = self.tokenizer.batch_encode_plus(
                [input_], max_length=self.max_len_input,padding='max_length',
                truncation = True,return_tensors="pt"
            )
            # tokenize targets
            tokenized_targets = self.tokenizer.batch_encode_plus(
                [target], max_length=self.max_len_output,padding='max_length',
                truncation = True,
                return_tensors="pt"
            )

            self.inputs.append(tokenized_inputs)
            self.targets.append(tokenized_targets)

In [ ]:
train_path = '/content/drive/MyDrive/train_race.parquet' # change this accordingly
validation_path = '/content/drive/MyDrive/validation_race.parquet'
train_dataset = QuestionGenerationDataset(t5_tokenizer,train_path)
validation_dataset = QuestionGenerationDataset(t5_tokenizer,validation_path)

In [ ]:
# Data Sample

train_sample = train_dataset[10] # thanks to __getitem__
decoded_train_input = t5_tokenizer.decode(train_sample['source_ids'])
decoded_train_output = t5_tokenizer.decode(train_sample['target_ids'])

print(decoded_train_input)
print(decoded_train_output)

# Fine Tuning T5

In [ ]:
import pytorch_lightning as pl
from torch.optim import AdamW
import argparse
from transformers import (
    get_linear_schedule_with_warmup
  )

class T5Tuner(pl.LightningModule):

    def __init__(self,t5model, t5tokenizer,batchsize=4):
        super().__init__()
        self.model = t5model
        self.tokenizer = t5tokenizer
        self.batch_size = batchsize

    def forward( self, input_ids, attention_mask=None,
                decoder_attention_mask=None,
                lm_labels=None):

         outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=decoder_attention_mask,
            labels=lm_labels,
        )

         return outputs

    def training_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_attention_mask=batch['target_mask'],
            lm_labels=batch['labels']
        )

        loss = outputs[0]
        self.log('train_loss',loss)
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_attention_mask=batch['target_mask'],
            lm_labels=batch['labels']
        )

        loss = outputs[0]
        self.log("val_loss",loss)
        return loss

    def train_dataloader(self):
        return DataLoader(train_dataset, batch_size=self.batch_size,
                          num_workers=2)

    def val_dataloader(self):
        return DataLoader(validation_dataset,
                          batch_size=self.batch_size,
                          num_workers=2)

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=3e-4, eps=1e-8)
        return optimizer

In [ ]:
model = T5Tuner(t5_model,t5_tokenizer)

trainer = pl.Trainer(max_epochs = 3,accelerator=device)

trainer.fit(model)

In [ ]:
# saving the model
!mkdir "t5_tokenizer"
!mkdir "t5_trained_model"
model.model.save_pretrained('t5_trained_model')
t5_tokenizer.save_pretrained('t5_tokenizer')

# Inference / Predictions

Run at this point

In [ ]:
trained_model_path = '/content/drive/MyDrive/t5_trained_model'
trained_tokenizer = '/content/drive/MyDrive/t5_tokenizer'
device = 'cpu'

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(trained_model_path)
tokenizer = T5Tokenizer.from_pretrained(trained_tokenizer)

Text Sample

In [ ]:
context ="President Donald Trump said and predicted that some states would reopen this month."
text = "context: "+context
print(text)

In [ ]:
context ="Holi is considered as one of the most revered and celebrated festivals of India and it is celebrated in almost every part of the country. It is also sometimes called as the “festival of love” as on this day people get to unite together forgetting all resentments and all types of bad feeling towards each other. The great Indian festival lasts for a day and a night, which starts in the evening of Purnima or the Full Moon Day in the month of Falgun. It is celebrated with the name Holika Dahan or Choti Holi on first evening of the festival and the following day is called Holi. In different parts of the country it is known with different names. The vibrancy of colors is something that brings in a lot of positivity in our lives and Holi being the festival of colours is actually a day worth rejoicing. Holi is a famous Hindu festival that is celebrated in every part of India with utmost joy and enthusiasm. The ritual starts by lighting up the bonfire one day before the day of Holi and this process symbolizes the triumph of good over the bad. On the day of Holi people play with colours with their friends and families and in evening they show love and respect to their close ones with Abeer."

text = "context: "+context
print(text)

In [ ]:
encoding = tokenizer.encode_plus(text,max_length =512,padding='max_length',
                                 truncation = True,
                                 return_tensors="pt").to(device)
print (encoding.keys())
input_ids,attention_mask  = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

In [ ]:
model.eval()

beam_outputs = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_length=72, # How long the generated questions should be
    early_stopping=True,
    num_beams=5,
    num_return_sequences=1
)

for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    print(sent)

# Deployment Demo

In [ ]:
!pip install --quiet gradio==3.9

In [ ]:
def get_question(sentence,mdl,tknizer):

  ''' function to generate questions. Takes a sentence,answer,
      model and tokenizer
  '''

  text = "context: {}".format(sentence)
  #print (text)
  max_len = 256
  encoding = tknizer.encode_plus(text,max_length=max_len, pad_to_max_length=False,truncation=True, return_tensors="pt")

  input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

  outs = mdl.generate(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  early_stopping=True,
                                  num_beams=5,
                                  num_return_sequences=1,
                                  no_repeat_ngram_size=2,
                                  max_length=72)
  #print("outputs")
  #print(outs)
  for beam_output in outs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    #print(sent)

  dec = [tknizer.decode(ids,skip_special_tokens=True) for ids in outs]
  #print("decs")
  #print(dec)

  Question = dec[0]
  #Question= Question.strip()
  #Answer= Answer.strip()
  #print(Answer)
  index = Question.find("answer:")

  # Extract the question and answer based on the position
  question = Question[10:index].strip()
  answer = Question[index + len("answer:"):].strip()
  #print("Question1:", question)
  #print("Answer1:", answer)

  return  question,answer

In [ ]:
context = "There had been a series of violent border skirmishes between the two countries after the 1959 Tibetan uprising, when India granted asylum to the Dalai Lama. Chinese military action grew increasingly aggressive after India rejected proposed Chinese diplomatic settlements throughout 1960–1962, with China resuming previously banned forward patrols in Ladakh after 30 April 1962. Amidst the Cuban Missile Crisis, China abandoned all attempts towards a peaceful resolution on 20 October 1962, invading disputed territory along the 3,225-kilometre (2,004 mi) border in Ladakh and across the McMahon Line in the northeastern frontier.[citation needed] Chinese troops pushed Indian forces back in both theatres, capturing all of their claimed territory in the western theatre and the Tawang Tract in the eastern theatre. The conflict ended when China unilaterally declared a ceasefire on 20 November 1962, and simultaneously announced its withdrawal to its pre-war position, the effective China–India border also known as the Line of Actual Control."

print("context: ",context)
ques,answer = get_question(context,model,tokenizer)
print ("question: ",ques)
print ("answer: ",answer)

context:  There had been a series of violent border skirmishes between the two countries after the 1959 Tibetan uprising, when India granted asylum to the Dalai Lama. Chinese military action grew increasingly aggressive after India rejected proposed Chinese diplomatic settlements throughout 1960–1962, with China resuming previously banned forward patrols in Ladakh after 30 April 1962. Amidst the Cuban Missile Crisis, China abandoned all attempts towards a peaceful resolution on 20 October 1962, invading disputed territory along the 3,225-kilometre (2,004 mi) border in Ladakh and across the McMahon Line in the northeastern frontier.[citation needed] Chinese troops pushed Indian forces back in both theatres, capturing all of their claimed territory in the western theatre and the Tawang Tract in the eastern theatre. The conflict ended when China unilaterally declared a ceasefire on 20 November 1962, and simultaneously announced its withdrawal to its pre-war position, the effective China–I

In [ ]:
context = '''Economic agents can be individuals, businesses, organizations, or governments. Economic transactions occur when two groups or parties agree to the value or price of the transacted good or service, commonly expressed in a certain currency. However, monetary transactions only account for a small part of the economic domain.

Economic activity is spurred by production which uses natural resources, labor and capital. It has changed over time due to technology, innovation (new products, services, processes, expanding markets, diversification of markets, niche markets, increases revenue functions) and changes in industrial relations (most notably child labor being replaced in some parts of the world with universal access to education).'''

print("context: ",context)
ques,answer = get_question(context,model,tokenizer)
print ("question: ",ques)
print ("answer: ",answer)

context:  Economic agents can be individuals, businesses, organizations, or governments. Economic transactions occur when two groups or parties agree to the value or price of the transacted good or service, commonly expressed in a certain currency. However, monetary transactions only account for a small part of the economic domain.

Economic activity is spurred by production which uses natural resources, labor and capital. It has changed over time due to technology, innovation (new products, services, processes, expanding markets, diversification of markets, niche markets, increases revenue functions) and changes in industrial relations (most notably child labor being replaced in some parts of the world with universal access to education).
question:  Are there any other interesting aspects about this article?
answer:  ['Economy agents can be individuals, businesses, organizations, or governments.']


Stripping paragraphs from paper

In [ ]:
!pip install PyPDF2


In [ ]:
text_input = """ The Sino-Indian War also known as the China–India War, Indo-China War, Indo-China War of 1962 or Sino-Indian War of 1962, was a military conflict between China and India that took place from October to November 1962. It was a military escalation of the Sino-Indian border dispute. Fighting occurred along India's border with China, in India's North-East Frontier Agency east of Bhutan, and in Aksai Chin west of Nepal.

There had been a series of violent border skirmishes between the two countries after the 1959 Tibetan uprising, when India granted asylum to the Dalai Lama. Chinese military action grew increasingly aggressive after India rejected proposed Chinese diplomatic settlements throughout 1960–1962, with China resuming previously banned "forward patrols" in Ladakh after 30 April 1962.[9][10] Amidst the Cuban Missile Crisis, China abandoned all attempts towards a peaceful resolution on 20 October 1962, invading disputed territory along the 3,225-kilometre (2,004 mi) border in Ladakh and across the McMahon Line in the northeastern frontier.[citation needed] Chinese troops pushed Indian forces back in both theatres, capturing all of their claimed territory in the western theatre and the Tawang Tract in the eastern theatre. The conflict ended when China unilaterally declared a ceasefire on 20 November 1962, and simultaneously announced its withdrawal to its pre-war position, the effective China–India border (also known as the "Line of Actual Control").

Much of the fighting comprised mountain warfare, entailing large-scale combat at altitudes of over 4,000 metres (13,000 feet).[citation needed] Notably, the war took place entirely on land, without the use of naval or air assets by either side.

As the Sino-Soviet split deepened, the Soviet Union made a major effort to support India, especially with the sale of advanced MiG fighter-aircraft. Simultaneously, the United States and the United Kingdom refused to sell advanced weaponry to India, further compelling it to turn to the Soviets for military aid.[11][12] """

In [ ]:
def strip_into_paragraphs(text):

    # Split the text into individual lines
    lines = text.split('\n')

    # Initialize an empty list to store paragraphs
    paragraphs = []

    # Initialize an empty string to store the current paragraph
    current_paragraph = ''

    # Iterate over each line
    for line in lines:
        # If the line is empty, consider it as a paragraph break
        if not line.strip():
            # Append the current paragraph to the list if it's not empty
            if current_paragraph.strip():
                paragraphs.append(current_paragraph.strip())
            # Reset the current paragraph
            current_paragraph = ''
        else:
            # Append the line to the current paragraph with a space
            current_paragraph += line.strip() + ' '

    # Append the last paragraph if it's not empty
    if current_paragraph.strip():
        paragraphs.append(current_paragraph.strip())

    # Return the list of paragraphs
    return paragraphs

# Example usage




In [ ]:
!pip install rouge_score
import nltk
nltk.download('punkt')
nltk.download('wordnet')
!pip install evaluate
!pip install mauve-text

Bleu & Meteor & Rougue between Context and Questions

In [ ]:
result = strip_into_paragraphs(text_input)
!pip install rouge-score
!pip install wordnet
from nltk import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
def U(K):
    i=0
    for paragraph in result :
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL' ], use_stemmer=True)
        i=i+1
        print("context: ",paragraph)
        ques,answer = get_question1(paragraph,model1,tokenizer1)
        print ("question: ",ques)
        print ("answer: ",answer)
        BLEU = sentence_bleu([paragraph], ques, weights=(0.2, 0.2, 0.2, 0.2))
        print(f"\nThe score for BLEU is: {BLEU}")
        scores = scorer.score(str(paragraph),
                      str(ques))
        print('rouge_score',scores)
        m_score=0
        for line in zip(paragraph, ques):
                ref = word_tokenize(line[0])
                #print(ref)
                hypo = word_tokenize(line[1])
                #print(hypo)
                m_score += nltk.translate.meteor_score.meteor_score([ref], hypo)
        print('Meteor Score:',m_score)
        print('---')
        if (i==K):
          break

Bleu & Meteor & Rougue between Human Generated Question and Machine Generated Questions

In [ ]:
! pip install  PyPDF2
import PyPDF2
!pip install pycryptodome
reader = PyPDF2.PdfReader('/content/drive/MyDrive/001-HIDE-AND-SEEK-Free-Childrens-Book-By-Monkey-Pen.pdf')

for pg in range(2,len(reader.pages)):
      #print("New Para: "+ str(pg))
      context=reader.pages[pg].extract_text()
      result = strip_into_paragraphs(context)
!pip install wordnet
from nltk import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
#import PyPDF2
#reader = PyPDF2.PdfReader('/content/drive/MyDrive/1411.1784.pdf')
def U(K):
    i=0
    for paragraph in result :
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL' ], use_stemmer=True)
        i=i+1
        print("context: ",paragraph)
        ques,answer = get_question1(paragraph,model1,tokenizer1)
        print ("question: ",ques)
        print ("answer: ",answer)
        reference = input("\nPlease enter the corresponding human generated question from the context (you can use Chatgpt for this) and confirm with Enter: ")
        BLEU = sentence_bleu([reference], ques, weights=(0.2, 0.2, 0.2, 0.2))
        print(f"\nThe score for BLEU is: {BLEU}")
        scores = scorer.score(str(reference),
                      str(ques))
        print('rouge_score',scores)
        m_score=0
        for line in zip(reference, ques):
                ref = word_tokenize(line[0])
                #print(ref)
                hypo = word_tokenize(line[1])
                #print(hypo)
                m_score += nltk.translate.meteor_score.meteor_score([ref], hypo)
        print('Meteor Score:',m_score)
        print('---')
        if (i==K):
          break

In [ ]:
! pip install PyPDF2
import re
import PyPDF2

# Open the PDF file'/content/drive/MyDrive/1411.1784.pdf'
pdf_file_path = '/content/drive/MyDrive/1411.1784.pdf'
with open(pdf_file_path, "rb") as file:
    pdf_reader = PyPDF2.PdfReader(file)
    num_pages = len(pdf_reader.pages)

    # Initialize an empty string to store the extracted text
    extracted_text = ""

    # Extract text from each page
    for page_num in range(num_pages):
        page = pdf_reader.pages[page_num]
        extracted_text += page.extract_text()

# Define regular expressions to match figures, tables, author names, and titles
figure_regex = r"\b(Fig(?:ure)?\.?\s\d+)\b"
table_regex = r"\b(Table\s\d+)\b"
author_regex = r"\bby\s[\w\s]+\b"
title_regex = r"^\s*[^\n]+\n"

# Remove figures, tables, author names, and titles

extracted_text = re.sub(figure_regex, "", extracted_text, flags=re.IGNORECASE)

extracted_text = re.sub(table_regex, "", extracted_text, flags=re.IGNORECASE)

extracted_text = re.sub(author_regex, "", extracted_text, flags=re.IGNORECASE)

#extracted_text = re.sub(title_regex, "", extracted_text, flags=re.MULTILINE)

# Remove extra spaces and line breaks
#extracted_text = re.sub(r"\s+", " ", extracted_text)

# Print the resulting extracted text'''
print(extracted_text)

In [ ]:
!pip install trafilatura
import trafilatura
downloaded = trafilatura.fetch_url('https://github.blog/2019-03-29-leader-spotlight-erin-spiceland/')
trafilatura.extract(downloaded)
# outputs main content and comments as plain text ...
trafilatura.extract(downloaded, output_format="xml", include_comments=False)

In [ ]:
U(1)

In [ ]:
import re
from PyPDF2 import PdfReader

def extract_sections_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PdfReader(file)
        sections = {
            'abstract': '',
            'introduction': '',
            'related_work': ''
        }
        current_section = None  # Variable to track the current section

        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text = page.extract_text()

            # Search for section headings using regular expressions
            if re.search(r"(?i)\babstract\b", text):
                current_section = 'abstract'
            if re.search(r"(?i)\bintroduction\b", text):
                current_section = 'introduction'
            #if re.search(r"(?i)\brelated work\b", text):
                #current_section = 'related_work'

            # If inside a desired section, append the text to the respective section variable
            if current_section in sections:
                sections[current_section] += text

            # Check if we have reached the end of the desired sections
            if current_section == 'related_work' and re.search(r"(?i)\bmethodology\b|\bexperimental results\b", text):
                break

        # Remove author information from sections
        author_regex = r"\bby\s[\w\s]+\b"
        for section in sections:
            sections[section] = re.sub(author_regex, "", sections[section], flags=re.IGNORECASE)

        return sections

# Example usage
#pdf_file_path = '/content/drive/MyDrive/1411.1784.pdf'
pdf_file_path = '/content/drive/MyDrive/s13369-023-07840-7 (1).pdf'
extracted_sections = extract_sections_from_pdf(pdf_file_path)

# Print the extracted sections
for section, content in extracted_sections.items():
    print(f"--- {section.capitalize()} ---")
    print(content)
    print('\n')



Data Pre Processing

In [ ]:
#context = "The Sino-Indian War also known as Indo-China War Indo-China War of 1962 or Sino-Indian War of 1962 took place between China and India from October to November 1962."
def ques_answer(self,n):
    import PyPDF2

    # creating a pdf reader object
    reader = PyPDF2.PdfReader('/content/drive/MyDrive/1411.1784.pdf')

    # print the number of pages in pdf file
    #print(len(reader.pages))

    # print the text of the first page
    #for pg in range(0,len(reader.pages)):
    for pg in range(0,n):
      #print("New Para: "+ str(pg))
      context=reader.pages[pg].extract_text()
      #print(context)
      ques, answer = get_question(context,model,tokenizer)
      #print ("question: ",ques)
      #print ("answer: ",answer)
    return context, ques,answer
    #return ques

In [ ]:
print(ques_answer(1,5))

In [ ]:
import gradio as gr

context = gr.inputs.Textbox(lines=5,placeholder="Enter the Context")
# answer = gr.inputs.Textbox(lines=3, placeholder="Enter answer/keyword here...")
question = gr.outputs.Textbox( type="auto", label="Question")
#answer = gr.outputs.Textbox( type="auto", label="Answer")

def ques_answer(context):
  return get_question(context,model,tokenizer)

iface = gr.Interface(
  fn=ques_answer,
  inputs=[context],
  outputs=question)


iface.launch(debug=False,share=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install PyPDF2

In [ ]:
import PyPDF2

# creating a pdf reader object
reader = PyPDF2.PdfReader('T5Paper_goodReference.pdf')

# print the number of pages in pdf file
print(len(reader.pages))

# print the text of the first page
for pg in range(0,len(reader.pages)):
  print("New Para: "+ str(pg))
  print(reader.pages[pg].extract_text())

In [ ]:
a=reader.pages[1].extract_text()
print(a)

In [ ]:
!pip install sentence_transformers

In [ ]:
"""
This is a simple application for sentence embeddings: clustering

Sentences are mapped to sentence embeddings and then k-mean clustering is applied.
"""
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans

embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Corpus with example sentences
#corpus = ['A man is eating food. A man is eating a piece of bread. A man is eating pasta.The girl is carrying a baby. The baby is carried by the womanA man is riding a horse. A man is riding a white horse on an enclosed ground. A monkey is playing drumsSomeone in a gorilla costume is playing a set of drums. A cheetah is running behind its prey. A cheetah chases prey on across a field.']
#for pg in range(0,len(reader.pages)):
for pg in range(0,5):
  #print("New Para: "+ str(pg))
    context=reader.pages[pg].extract_text()
    corpus1 = context.split('. ')
    #print(corpus1)
    corpus_embeddings = embedder.encode(corpus1)
    num_clusters = 5
    clustering_model = KMeans(n_clusters=num_clusters)
    clustering_model.fit(corpus_embeddings)
    cluster_assignment = clustering_model.labels_
    #print(cluster_assignment)
    clustered_sentences = [[] for i in range(num_clusters)]
    for sentence_id, cluster_id in enumerate(cluster_assignment):
      clustered_sentences[cluster_id].append(corpus1[sentence_id])
      #print(clustered_sentences)

    for i, cluster in enumerate(clustered_sentences):
      print("Context ", i+1)
      print(cluster)
      print("")
      ques, answer = get_question(cluster,model,tokenizer)
      print ("question: ",ques)
      print ("answer: ",answer)


Clustering & Question/Answer Pair Generation

#Get accuracy Percentage, what optimisation & strategies needs to be adopted to improve the system

In [ ]:
def ClusQuesGen(a,b,n):
#a=1st page of the book, b=last page of the pdf, n=number of questions you want from each page
      import PyPDF2
      from sentence_transformers import SentenceTransformer
      from sklearn.cluster import KMeans

      embedder = SentenceTransformer('all-MiniLM-L6-v2')
      reader = PyPDF2.PdfReader('/content/drive/MyDrive/2006.09595 (1).pdf')

      # Corpus with example sentences
      #corpus = ['A man is eating food. A man is eating a piece of bread. A man is eating pasta.The girl is carrying a baby. The baby is carried by the womanA man is riding a horse. A man is riding a white horse on an enclosed ground. A monkey is playing drumsSomeone in a gorilla costume is playing a set of drums. A cheetah is running behind its prey. A cheetah chases prey on across a field.']
      #for pg in range(0,len(reader.pages)):
      ques_answer=[]
      for pg in range(a,b):
        #print("New Para: "+ str(pg))
          context=reader.pages[pg].extract_text()
          corpus1 = context.split('. ')
          #print(corpus1)
          corpus_embeddings = embedder.encode(corpus1)
          num_clusters = n
          clustering_model = KMeans(n_clusters=num_clusters)
          clustering_model.fit(corpus_embeddings)
          cluster_assignment = clustering_model.labels_
          #print(cluster_assignment)
          clustered_sentences = [[] for i in range(num_clusters)]
          for sentence_id, cluster_id in enumerate(cluster_assignment):
            clustered_sentences[cluster_id].append(corpus1[sentence_id])
            #print(clustered_sentences)
          for i, cluster in enumerate(clustered_sentences):
            print("Context ", i+1)
            print(cluster)
            print("")
            ques, answer = get_question(cluster,model,tokenizer)
            ques_answer.append([cluster, i+1, ques, answer])
            #return cluster, ques, answer
            #print(ques_answer)
            print ("question: ",ques)
            print ("answer: ",answer)
      #return cluster, ques, answer


In [ ]:
ClusQuesGen(3,4,2)


In [ ]:
import gradio as gr

#context = gr.inputs.Textbox(lines=5,placeholder="Enter paragraph/context here...")
# answer = gr.inputs.Textbox(lines=3, placeholder="Enter answer/keyword here...")
a = gr.inputs.Textbox(lines=5,placeholder="Enter starting page number of the text:")
b = gr.inputs.Textbox(lines=5,placeholder="Enter ending page number of the text:")
n = gr.inputs.Textbox(lines=5,placeholder="Enter number of questions needs to be generated from each page:")
#context = gr.outputs.Textbox( type="auto", label="cluster")
#question = gr.outputs.Textbox( type="auto", label="Question")
#answer = gr.outputs.Textbox( type="auto", label="Answer")

#def generate_question(a,b,n):
def generate_question(context):
  return get_question(cluster,model,tokenizer)
  #return ClusQuesGen(a,b,n)

iface = gr.Interface(
  fn=generate_question,
  inputs=[a,b,n],
  outputs=["text", "text", "text"])
  outputs=["text"])
iface.launch(debug=False,share=True)

In [ ]:
import gradio as gr

context = gr.inputs.Textbox(lines=5,placeholder="Enter paragraph/context here...")
# answer = gr.inputs.Textbox(lines=3, placeholder="Enter answer/keyword here...")
question = gr.outputs.Textbox( type="auto", label="Question")
#answer = gr.outputs.Textbox( type="auto", label="Answer")

def generate_question(context):
  return get_question(context,model,tokenizer)

iface = gr.Interface(
  fn=generate_question,
  inputs=[context],
  outputs=question)
  #outputs=answer)

iface.launch(debug=False,share=True)

In [ ]:
import gradio as gr

def greet(name, is_morning, temperature):
    salutation = "Good morning" if is_morning else "Good evening"
    greeting = f"{salutation} {name}. It is {temperature} degrees today"
    celsius = (temperature - 32) * 5 / 9
    return greeting, round(celsius, 2)

demo = gr.Interface(
    fn=greet,
    inputs=["text", "checkbox", gr.Slider(0, 100)],
    outputs=["text", "number"],
)
demo.launch()

Scores on different Metrics

In [ ]:
!pip install rouge_score
import nltk
nltk.download('punkt')
nltk.download('wordnet')
!pip install evaluate
!pip install mauve-text

In [ ]:
import nltk
nltk.download('punkt')
#Bleu Score
from nltk.translate.bleu_score import sentence_bleu
reference = [
    'this is a dog'.split(),
    'it is dog'.split(),
    'dog it is'.split(),
    'a dog, it is'.split()
]
#reference
candidate = 'it is dog'.split()
print('BLEU score -> {}'.format(sentence_bleu(reference, candidate )))

candidate = 'it is a dog'.split()
print('BLEU score -> {}'.format(sentence_bleu(reference, candidate)))

#rouge_scorer
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
'''scores = scorer.score('The quick brown fox jumps over the lazy dog',
                      'The quick brown dog jumps on the log.')'''
scores = scorer.score('On which factors does the economy functions?',
                      'What is the social domain of an economy')
print('rouge_score',scores)


#meteor score
import nltk
from nltk.tokenize import word_tokenize
refs=["On which factors does the economy functions?"]
hypos=["What is the social domain of an economy"]
m_score=0
for line in zip(refs, hypos):
        ref = word_tokenize(line[0])
        hypo = word_tokenize(line[1])
        m_score += nltk.translate.meteor_score.meteor_score([ref], hypo)
print('Meteor Score:',m_score)


#Mauve Score
from evaluate import load
mauve = load('mauve')
#predictions = ["hello world", "goodnight moon"]
#references = ["hello there", "general kenobi"]
predictions = ["On which factors does the economy functions?"]
references = ["What is the social domain of an economy"]
mauve_results = mauve.compute(predictions=predictions, references=references)
print('Mauve:',mauve_results.mauve)

In [ ]:
#Import the word_tokenize and sentence_bleu methods from NLTK.
from nltk import word_tokenize
from nltk.translate.bleu_score import sentence_bleu

#Enter the machine-translated sentence (hypothesis) and the human reference translation (reference).
hypothesis = input("\nPlease enter a machine-translated sentence and confirm with Enter: ")
reference = input("\nPlease enter the corresponding human reference translation and confirm with Enter: ")

#Tokenize the two sentences and store the individual tokes in two lists (one for the hypothesis tokens and one for the reference tokens).
hypothesis = word_tokenize(hypothesis)
reference = word_tokenize(reference)

#Calculate and print BLEU-1, using 1-grams as highest-order n-grams
#Reference is placed in [square brackets] because you can score the machine-translated sentence against multiple references (applies to all BLEU score calculations listed here).
#The weights are set so that calculation is based solely on 1-gram precision.
BLEU_1 = sentence_bleu([reference], hypothesis, weights=(1, 0, 0, 0))
print(f"\nThe score for BLEU-1 is: {BLEU_1}")

#Calculate and print BLEU-2, using 2-grams as highest-order n-grams.
#The weights are set so that calculation is based on 1-gram and 2-gram precisions.
BLEU_2 = sentence_bleu([reference], hypothesis, weights=(0.5, 0.5, 0, 0))
print(f"\nThe score for BLEU-2 is: {BLEU_2}")

#Calculate and print BLEU-3, using 3-grams as highest-order n-grams.
#The weights are set so that calculation is based on 1-gram, 2-gram and 3-gram precisions.
BLEU_3 = sentence_bleu([reference], hypothesis, weights=(0.33, 0.33, 0.33, 0))
print(f"\nThe score for BLEU-3 is: {BLEU_3}")

#Calculate and print BLEU-4, using 4-grams as highest-order n-grams.
#The weights are set so that calculation is based on 1-gram, 2-gram, 3-gram and 4-gram precisions.
BLEU_4 = sentence_bleu([reference], hypothesis, weights=(0.25, 0.25, 0.25, 0.25))
print(f"\nThe score for BLEU-4 is: {BLEU_4}")

#Calculate and print BLEU-5, using 5-grams as highest-order n-grams.
#The weights are set so that calculation is based on 1-gram, 2-gram, 3-gram, 4-gram and 5-gram precisions.
BLEU_5 = sentence_bleu([reference], hypothesis, weights=(0.2, 0.2, 0.2, 0.2, 0.2))
print(f"\nThe score for BLEU-5 is: {BLEU_5}")

Word 2 Vec

In [ ]:
!pip install word2vec
!pip install gensim

In [ ]:
sentences='this is a dog'

In [ ]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
sentence = "this is an example sentence."
tokenized_sentence = simple_preprocess(sentence)
#print(tokenized_sentence)
model = Word2Vec(sentences, min_count=1)
vector = model.wv["This"]
similarity_score = model.wv.similarity("This#", "sentence")
print(similarity_score)

In [ ]:
def create_dataset(filename,  verbose = False):
  data  = pd.read_csv(filename, skipinitialspace=True, usecols=['Context','Question','Answer'] )
  result_df1  = pd.DataFrame(columns = ['context', 'question','answer'])
  result_df1 = data[0:177, : ]
  return result_df1

In [ ]:
df3

In [ ]:
filename = r"Dataset - Sheet1.csv"

new_df = create_dataset(filename)
#new_df = create_dataset(df3)
print(new_df)

In [ ]:
df3

In [ ]:
class QuestionGenerationDataset_New(Dataset):
    def __init__(self, tokenizer, data, max_len_inp=512,max_len_out=512):
        self.passage_column = "Context"
        self.answer = "Answer"
        self.question = "Question"

        # self.data = pd.read_csv(self.path)
        self.data = data
        self.max_len_input = max_len_inp
        self.max_len_output = max_len_out
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []
        self._build()

    def __len__(self):
        return len(self.inputs)


    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask = self.inputs[index]["attention_mask"].squeeze()  #squeeze to get rid of the batch dimension
        target_mask = self.targets[index]["attention_mask"].squeeze()  # convert [batch,dim] to [dim]


        labels = copy.deepcopy(target_ids)
        labels [labels==0] = -100

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask,"labels":labels}

    def _build(self):
        for rownum,val in tqdm(self.data.iterrows()): # Iterating over the dataframe

            passage,answer,target = val[self.passage_column],val[self.answer],val[self.question]

            input_ = f"context: {passage}" # T5 Input format for question answering tasks
            target = f"question: {str(target)} answer: {answer}" # Output format we require
            # tokenize inputs
            tokenized_inputs = self.tokenizer.batch_encode_plus(
                [input_], max_length=self.max_len_input,padding='max_length',
                truncation = True,return_tensors="pt"
            )
            # tokenize targets
            tokenized_targets = self.tokenizer.batch_encode_plus(
                [target], max_length=self.max_len_output,padding='max_length',
                truncation = True,
                return_tensors="pt"
            )

            self.inputs.append(tokenized_inputs)
            self.targets.append(tokenized_targets)

In [ ]:
train_dataset1 = QuestionGenerationDataset_New(t5_tokenizer,new_df.sample(frac = 0.7))
validation_dataset1 = QuestionGenerationDataset_New(t5_tokenizer,new_df.drop(new_df.sample(frac = 0.7).index))

In [ ]:
train_dataset1 = QuestionGenerationDataset_New(t5_tokenizer,df3.sample(frac = 0.7))
validation_dataset1 = QuestionGenerationDataset_New(t5_tokenizer,df3.drop(df3.sample(frac = 0.7).index))

In [ ]:
import pytorch_lightning as pl
from torch.optim import AdamW
import argparse
from transformers import (
    get_linear_schedule_with_warmup
  )

class T5Tuner1(pl.LightningModule):

    def __init__(self,t5model, t5tokenizer,batchsize=4):
        super().__init__()
        self.model = t5model
        self.tokenizer = t5tokenizer
        self.batch_size = batchsize

    def forward( self, input_ids, attention_mask=None,
                decoder_attention_mask=None,
                lm_labels=None):

         outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=decoder_attention_mask,
            labels=lm_labels,
        )

         return outputs

    def training_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_attention_mask=batch['target_mask'],
            lm_labels=batch['labels']
        )

        loss = outputs[0]
        self.log('train_loss',loss)
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_attention_mask=batch['target_mask'],
            lm_labels=batch['labels']
        )

        loss = outputs[0]
        self.log("val_loss",loss)
        return loss

    def train_dataloader(self):
        return DataLoader(train_dataset1, batch_size=self.batch_size,
                          num_workers=2)

    def val_dataloader(self):
        return DataLoader(validation_dataset1,
                          batch_size=self.batch_size,
                          num_workers=2)

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=3e-4, eps=1e-8)
        return optimizer

In [ ]:
trained_model_path = '/content/drive/MyDrive/t5_trained_model'
trained_tokenizer = '/content/drive/MyDrive/t5_tokenizer'
device = 'cpu'

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(trained_model_path)
tokenizer = T5Tokenizer.from_pretrained(trained_tokenizer)

model1 = T5Tuner1(model,tokenizer)

trainer = pl.Trainer(max_epochs = 10,accelerator=device, log_every_n_steps=10)

trainer.fit(model1)

In [ ]:
# saving the model
#!mkdir "t5_tokenizer1"
#!mkdir "t5_trained_model1"
model1.model.save_pretrained('t5_trained_model1')
t5_tokenizer.save_pretrained('t5_tokenizer1')

10 Epochs

In [ ]:
model1.model.save_pretrained('/content/drive/MyDrive/t5_trained_model2')
t5_tokenizer.save_pretrained('/content/drive/MyDrive/t5_tokenizer2')

In [ ]:
model1.model.save_pretrained('/content/drive/MyDrive/t5_trained_model1')
t5_tokenizer.save_pretrained('/content/drive/MyDrive/t5_tokenizer1')

Inference

In [ ]:
trained_model_path1 = '/content/drive/MyDrive/t5_trained_model2'
trained_tokenizer1 = '/content/drive/MyDrive/t5_tokenizer2'
device = 'cpu'

In [ ]:
model1 = T5ForConditionalGeneration.from_pretrained(trained_model_path1)
tokenizer1 = T5Tokenizer.from_pretrained(trained_tokenizer1)

In [ ]:
def get_question1(sentence,mdl,tknizer):

  ''' function to generate questions. Takes a sentence,answer,
      model and tokenizer
  '''

  text = "context: {}".format(sentence)
  #print (text)
  max_len = 256
  encoding = tknizer.encode_plus(text,max_length=max_len, pad_to_max_length=False,truncation=True, return_tensors="pt")

  input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

  outs = mdl.generate(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  early_stopping=True,
                                  num_beams=5,
                                  num_return_sequences=1,
                                  no_repeat_ngram_size=2,
                                  max_length=72)
  #print("outputs")
  #print(outs)
  for beam_output in outs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    #print(sent)

  dec = [tknizer.decode(ids,skip_special_tokens=True) for ids in outs]
  #print("decs")
  #print(dec)

  Question = dec[0]
  #Question= Question.strip()
  #Answer= Answer.strip()
  #print(Answer)
  index = Question.find("answer:")

  # Extract the question and answer based on the position
  question = Question[10:index].strip()
  answer = Question[index + len("answer:"):].strip()
  #print("Question1:", question)
  #print("Answer1:", answer)

  return  question,answer

In [ ]:
context = "The Sino-Indian War also known as the China–India War, Indo-China War, Indo-China War of 1962 or Sino-Indian War of 1962, was a military conflict between China and India that took place from October to November 1962. It was a military escalation of the Sino-Indian border dispute. Fighting occurred along India's border with China, in India's North-East Frontier Agency east of Bhutan, and in Aksai Chin west of Nepal."

print("context: ",context)
ques,answer = get_question1(context,model1,tokenizer1)
print ("question: ",ques)
print ("answer: ",answer)

context:  The Sino-Indian War also known as the China–India War, Indo-China War, Indo-China War of 1962 or Sino-Indian War of 1962, was a military conflict between China and India that took place from October to November 1962. It was a military escalation of the Sino-Indian border dispute. Fighting occurred along India's border with China, in India's North-East Frontier Agency east of Bhutan, and in Aksai Chin west of Nepal.
question:  What was the military escalation of the Sino-Indian border dispute between China and India?
answer:  The conflict occurred along India's border with China, in India’s North-East Frontier Agency east of Bhutan, and in Aksai Chin west of Nepal.


In [ ]:
context = '''Economic agents can be individuals, businesses, organizations, or governments. Economic transactions occur when two groups or parties agree to the value or price of the transacted good or service, commonly expressed in a certain currency. However, monetary transactions only account for a small part of the economic domain.

Economic activity is spurred by production which uses natural resources, labor and capital. It has changed over time due to technology, innovation new products, services, processes, expanding markets, diversification of markets, niche markets, increases revenue functions and changes in industrial relations most notably child labor being replaced in some parts of the world with universal access to education.'''

print("context: ",context)
ques,answer = get_question1(context,model1,tokenizer1)
print ("question: ",ques)
print ("answer: ",answer)

context:  Economic agents can be individuals, businesses, organizations, or governments. Economic transactions occur when two groups or parties agree to the value or price of the transacted good or service, commonly expressed in a certain currency. However, monetary transactions only account for a small part of the economic domain.

Economic activity is spurred by production which uses natural resources, labor and capital. It has changed over time due to technology, innovation new products, services, processes, expanding markets, diversification of markets, niche markets, increases revenue functions and changes in industrial relations most notably child labor being replaced in some parts of the world with universal access to education.
question:  How does the economic domain change?
answer:  Economic activity is spurred by production which uses natural resources, labor and capital. It has changed over time due to technology, innovation new products, services, processes, expanding marke

In [ ]:
context='''1.2 DESIGNING A LEARNING SYSTEM
In order to illustrate some of the basic design issues and approaches to machine
learning, let us consider designing a program to learn to play checkers, with
the goal of entering it in the world checkers tournament. We adopt the obvious
performance measure: the percent of games it wins in this world tournament.
1.2.1 Choosing the Training Experience
The first design choice we face is to choose the type of training experience from
which our system will learn. The type of training experience available can have a
significant impact on success or failure of the learner. One key attribute is whether
the training experience provides direct or indirect feedback regarding the choices
made by the performance system. For example, in learning to play checkers, the
system might learn from direct training examples consisting of individual checkers
board states and the correct move for each. Alternatively, it might have available
only indirect information consisting of the move sequences and final outcomes
of various games played. In this later case, information about the correctness
of specific moves early in the game must be inferred indirectly from the fact
that the game was eventually won or lost. Here the learner faces an additional
problem of credit assignment, or determining the degree to which each move in
the sequence deserves credit or blame for the final outcome. Credit assignment can
be a particularly difficult problem because the game can be lost even when early
moves are optimal, if these are followed later by poor moves. Hence, learning from
direct training feedback is typically easier than learning from indirect feedback.
A second important attribute of the training experience is the degree to which
the learner controls the sequence of training examples. For example, the learner
might rely on the teacher to select informative board states and to provide the
correct move for each. Alternatively, the learner might itself propose board states
that it finds particularly confusing and ask the teacher for the correct move. Or the
learner may have complete control over both the board states and (indirect) training
classifications, as it does when it learns by playing against itself with no teacher
present. Notice in this last case the learner may choose between experimenting
with novel board states that it has not yet considered, or honing its skill by playing
minor variations of lines of play it currently finds most promising. Subsequent
chapters consider a number of settings for learning, including settings in which
training experience is provided by a random process outside the learner's control,
settings in which the learner may pose various types of queries to an expert teacher,
and settings in which the learner collects training examples by autonomously
exploring its environment.
A third important attribute of the training experience is how well it represents the distribution of examples over which the final system performance P must
be measured. In general, learning is most reliable when the training examples follow a distribution similar to that of future test examples. In our checkers learning
scenario, the performance metric P is the percent of games the system wins in
the world tournament. If its training experience E consists only of games played
against itself, there is an obvious danger that this training experience might not
be fully representative of the distribution of situations over which it will later be
tested. For example, the learner might never encounter certain crucial board states
that are very likely to be played by the human checkers champion. In practice,
it is often necessary to learn from a distribution of examples that is somewhat
different from those on which the final system will be evaluated (e.g., the world
checkers champion might not be interested in teaching the program!). Such situations are problematic because mastery of one distribution of examples will not
necessary lead to strong performance over some other distribution. We shall see
that most current theory of machine learning rests on the crucial assumption that
the distribution of training examples is identical to the distribution of test examples. Despite our need to make this assumption in order to obtain theoretical
results, it is important to keep in mind that this assumption must often be violated
in practice.
To proceed with our design, let us decide that our system will train by
playing games against itself. This has the advantage that no external trainer need
be present, and it therefore allows the system to generate as much training data
as time permits. We now have a fully specified learning task. '''
ques,answer = get_question1(context,model1,tokenizer1)
print ("question: ",ques)
print ("answer: ",answer)


question:  What are some basic design issues and approaches to machine learning?
answer:  Choosing the Training Experience The first design choice we face is to choose the type of training experience from which the system will learn. The design experience provides direct or indirect feedback regarding the choices made by the performance system. Alternatively, it may have available only indirect information consisting


In [ ]:
#context = "According to one embodiment of the present dis closure , a question - answering learning method is prtences among N sentences , wherein the classifier module includes several classifiers each representing different question - answering models , and both of the N and the N1 are positive integers . At least one corresponding sentence type of each of the N2 unlabeled sentences among the N sentences is determined by each of the classifiers , wherein the N2 is a positive integer . In a consistency evaluation process , N3 sentences are selected from the N2 unlabeled sentences by a consistency evaluation module according to a degree of consistency of determined results of the classifiers , wherein the determined results of the N3 sentences are determined by the classifiers and are inconsistent , and the N3 is a positive integer . In a complementarity evaluation process , N4 mutually complementary sentences are selected as to - be - labeled sentences from the N3 sentences by a complementarity evaluation module , wherein the N4 is a positive integer . After the N4 selected to - be - labeled sentences are labeled , the classifiers of the classifier module are re - created by the classifier generation module according to the N1 labeled sentences and the N4 selected to - be - labeled sentences . At least one of previously created classifiers is added to the classifier module to be members of the classifier module by a classifier evaluation module. According to another embodiment of the present disclosure , a question - answering learning system is provided . The question - answering learning system includes a classifier generation module , a consistency evaluation module , a complementarity evaluation module and a classifier evaluation module . The classifier generation module is configured to : create a classifier module according to N1 labeled sentences among N sentences , wherein the classifier module includes several classifiers each representing different ques tion - answering models , and both of the N and the N1 are positive integers . Each of the classifiers determines at least one corresponding sentence type of each of the N2 unlabeled sentences among the N sentences , wherein the N2 is a positive integer . The consistency evaluation module is configured to : select , in a consistency evaluation process , N3 sentences from the N2 unlabeled sentences according to a degree of consistency of determined results of the classifiers , wherein the determined results of the N3 sentences are determined by the classifiers and are inconsistent , and the N3 is a positive integer . "
#context = "In a complementarity evaluation process , N4 mutually complementary sentences are selected as to - be - labeled sentences from the N3 sentences by a complementarity evaluation module , wherein the N4 is a positive integer . After the N4 selected to - be - labeled sentences are labeled , the classifiers of the classifier module are re - created by the classifier generation module according to the N1 labeled sentences and the N4 selected to - be - labeled sentences . At least one of previously created classifiers is added to the classifier module to be members of the classifier module by a classifier evaluation module. According to another embodiment of the present disclosure , a question - answering learning system is provided . The question - answering learning system includes a classifier generation module , a consistency evaluation module , a complementarity evaluation module and a classifier evaluation module . The classifier generation module is configured to : create a classifier module according to N1 labeled sentences among N sentences , wherein the classifier module includes several classifiers each representing different ques tion - answering models , and both of the N and the N1 are positive integers . Each of the classifiers determines at least one corresponding sentence type of each of the N2 unlabeled sentences among the N sentences , wherein the N2 is a positive integer . The consistency evaluation module is configured to : select , in a consistency evaluation process , N3 sentences from the N2 unlabeled sentences according to a degree of consistency of determined results of the classifiers , wherein the determined results of the N3 sentences are determined by the classifiers and are inconsistent , and the N3 is a positive integer . "
#context = "Each of the classifiers determines at least one corresponding sentence type of each of the N2 unlabeled sentences among the N sentences , wherein the N2 is a positive integer . The consistency evaluation module is configured to : select , in a consistency evaluation process , N3 sentences from the N2 unlabeled sentences according to a degree of consistency of determined results of the classifiers , wherein the determined results of the N3 sentences are determined by the classifiers and are inconsistent , and the N3 is a positive integer . The complementarity evaluation module is configured to : select , in a complementarity evalu ation process , N4 mutually complementary sentences as to - be - labeled sentences from the N3 sentences , wherein the N4 is a positive integer . The classifier generation module is further configured to : re - create , after the N4 selected to - be labeled sentences are labeled , the classifiers of the classifier module according to the N1 labeled sentences and the N4 selected to - be - labeled sentences . The classifier evaluation module is configured to : add at least one of previously created classifiers to the classifier module to be members of the classifier module According to an alternate embodiment of the pres ent disclosure , a computer programming product is disclosed . The computer program product is installed in a question - answering learning system to execute a question answering learning method . The question - answering learning method includes the following : "
#context='omplementary sentences are selected as to - be - labeled sentences from the N3 sentences by a complementarity evaluation module , wherein the words or textual meanings of the N4 sentences are not mutually repetitive , similar , implicative or derivative , and the N4 is a positive integer . After the N4 selected to - be - labeled sentences are labeled , the classifiers of the classifier module are re - created by the classifier generation module according to the N1 labeled sentences and the N4 selected to - be - labeled sentences . At least one of previously created classifiers is added to the classifier module to be members of the classifier module by a classifier evaluation module The above and other aspects of the invention will become better understood with regards to the following detailed description of the preferred but non - limiting embodiment ( s ) . The following description is made with reference to the accompanying drawings'
context='''We present QuAC, a dataset for Question Answering in Context that contains 14K information-seeking QA dialogs (100K questions in total). The dialogs involve two crowd workers: (1) a student who poses a sequence of freeform questions to learn as much as possible about a hidden Wikipedia text, and (2) a teacher who answers the questions by providing short excerpts from the text. QuAC introduces challenges not found in existing machine comprehension datasets: its questions are often more open-ended, unanswerable, or only meaningful within the dialog context, as we show in a detailed qualitative evaluation. We also report results for a number of reference models, including a recently state-of-the-art reading comprehension architecture extended to model dialog context. Our best model underperforms humans by 20 F1, suggesting that there is significant room for future work on this data.
Dataset, baseline, and leaderboard available at http://quac.ai '''
#print("context: ",context)

ques,answer = get_question1(context,model1,tokenizer1)
print ("question: ",ques)
print ("answer: ",answer)

question:  What are the challenges presented in the QuAC dataset?
answer:  The challenges encountered in existing machine comprehension datasets include open-ended, unanswerable, or meaningful within the dialog context.


In [ ]:
context='''Our core evaluation metric, word-level F1, is implemented similarly to SQuAD : precision and recall are computed by considering the portion of words in the prediction and references that overlap after removing stopwords.12 For no answer questions, we give the system an F1 of one if it correctly predicts no answer and zero otherwise.13 Like SQuAD, we compute the maximum F1 among all references; however, since many questions have multiple valid answers, this metric varies significantly with the number of reference annotations. To make oracle human and system performance comparable, given n references, we report the average of the maximum F1 computed from each n − 1 subset with respect to the heldout reference. Additionally, since averaged F1 can be misleading for questions with multiple valid answers, we introduce the human equivalence score (HEQ), a performance measure for judging whether a system’s output is as good as that of an average human.14 HEQ measures the percentage of examples for which system F1 exceeds or matches human F1. We compute two variants: (1) the percentage of questions for which this is true (HEQ-Q), and (2) the percentage of dialogs for which this is true for every question in the dialog (HEQ-D). A system that achieves a value of 100 on HEQ-D can by definition maintain average human quality output over full dialogs. For dialog acts, we report accuracy with respect to the majority annotation, breaking ties randomly.'''
ques,answer = get_question1(context,model1,tokenizer1)
print ("question: ",ques)
print ("answer: ",answer)

question:  How does the human equivalence score (HEQ) measure the performance of a system's output as good as that of an average human?
answer:  For questions with multiple valid answers, the metric varies significantly with the number of reference annotations. To make oracle human and system performance comparable, given


In [ ]:
context='''Sanity check Overall, the poor sanity check results imply that is very challenging. Of these, following the transition matrix (TM) gives the best performance, reinforcing the observation that the dialog context plays a significant role in the task. Upper bounds The human upper bound (80.8 F1) demonstrates high agreement. While Gold sentence + NA does perform well, indicating that significant progress can be made by treating the problem as answer sentence selection, HEQ measures show that span-based approaches will be needed achieve average human equivalence. Finally, the Gold NA + TM shows that cannot be solved by ignoring question and answer text Baselines Text similarity methods such as bagof-ngrams overlap and InferSent are largely ineffective on , which shows that questions have little direct overlap with their answers. On the other hand, BiDAF++ models make significant progress, demonstrating that existing models can already capture a significant portion of phenomena in . The addition of information from previous turns (w/ 1-ctx) helps significantly, indicating that integration of context is essential to solving the task. While increasing the context size in BiDAF++ continues to help, we observe saturation using contexts of length 3, suggesting that more sophisticated models are necessary to take full advantage of the context. Finally, even our best model underperforms humans:
the system achieves human equivalence on only 60% of questions and 5% of full dialogs.'''
ques,answer = get_question1(context,model1,tokenizer1)
print ("question: ",ques)
print ("answer: ",answer)

question:  What does the poor results obtained from the sanity check imply about the QuAC task?
answer:  The lack of results from previous turns (w/ 1-ctx) helps significantly, indicating that integration of context is essential to solving the task. While increasing the context size in BiDAF++ continues to help, the


In [ ]:
def divide_into_paragraphs(text, num_paragraphs):
    # Split the text into paragraphs based on the desired number
    words = text.split()
    words_per_paragraph = len(words) // num_paragraphs

    paragraphs = []
    start_index = 0
    for _ in range(num_paragraphs - 1):
        end_index = start_index + words_per_paragraph
        paragraph = " ".join(words[start_index:end_index])
        paragraphs.append(paragraph)
        start_index = end_index

    # Add the remaining words as the last paragraph
    last_paragraph = " ".join(words[start_index:])
    paragraphs.append(last_paragraph)

    return paragraphs


In [ ]:
!pip install PyPDF2
import PyPDF2
import math

def qa_func(start_page: int, end_page: int, num_qa: int):
  #qa_dict=dict.fromkeys('Question','Answer')
  qa_dict={}
  num_qa_per_pg = math.ceil(num_qa/(end_page-start_page+1))
  if (num_qa_per_pg>5): #5 is hyperparamer which means it will generate 5 questions per page
    print('Enter less number of questions')
  else:

      pdf_file_path = "/content/drive/MyDrive/Machine Learning - Tom Mitchell_compressed.pdf"
      with open(pdf_file_path, "rb") as file:
          pdf_reader = PyPDF2.PdfReader(file)

          # Find the start and end page numbers of the desired chapter from the index
          # start_page = 10  # Replace with the actual start page number of the chapter
          # end_page = 20    # Replace with the actual end page number of the chapter

          # Extract pages of the desired chapter
          extracted_pages = []
          count=0
          for page_num in range(start_page, end_page+1):
              page = pdf_reader.pages[page_num].extract_text()
              #print(page_num)

              paragraphs = divide_into_paragraphs(page, num_qa_per_pg)
              for i, paragraph in enumerate(paragraphs):

                if (count<num_qa):
                  ques,answer = get_question1(paragraph,model1,tokenizer1)
                  print(paragraph)
                  qa_dict[ques] = answer
                  '''print ("question: ",ques)
                  print ("answer: ",answer)'''
                  count=count+1
                else:
                  #print ("qa_dict: ",qa_dict)
                  break
  return qa_dict
my_dict={}
my_dict=qa_func(14,15,5)
print(my_dict)

CHAPTER 1 INTRODUCITON 3 0 Learning to recognize spoken words. All of the most successful speech recognition systems employ machine learning in some form. For example, the SPHINX system (e.g., Lee 1989) learns speaker-specific strategies for recognizing the primitive sounds (phonemes) and words from the observed speech signal. Neural network learning methods (e.g., Waibel et al. 1989) and methods for learning hidden Markov models (e.g., Lee 1989) are effective for automatically customizing to,individual speakers, vocabularies, microphone characteristics, background noise, etc. Similar techniques have potential applications in many signal-interpretation problems. 0 Learning to drive an autonomous vehicle. Machine learning methods have been used to train computer-controlled vehicles to steer correctly when driving on a variety of road types. For example, the ALVINN system (Pomerleau 1989) has used its learned strategies to drive unassisted at 70 miles per hour
for 90 miles on public high

Modification with Vishu's Code

In [ ]:
!pip install PyPDF2
import PyPDF2
import math
import re

def qa_func(start_page: int, end_page: int, num_qa: int):
  #qa_dict=dict.fromkeys('Question','Answer')
  qa_dict={}
  figure_regex = r"\b(Fig(?:ure)?\.?\s\d+)\b"
  table_regex = r"\b(Table\s\d+)\b"
  num_qa_per_pg = math.ceil(num_qa/(end_page-start_page+1))
  if (num_qa_per_pg>5): #5 is hyperparamer which means it will generate 5 questions per page
    print('Enter less number of questions')
  else:

      pdf_file_path = "/content/drive/MyDrive/Machine Learning - Tom Mitchell_compressed.pdf"
      with open(pdf_file_path, "rb") as file:
          pdf_reader = PyPDF2.PdfReader(file)

          # Find the start and end page numbers of the desired chapter from the index
          # start_page = 10  # Replace with the actual start page number of the chapter
          # end_page = 20    # Replace with the actual end page number of the chapter

          # Extract pages of the desired chapter
          extracted_pages = []
          count=0
          for page_num in range(start_page, end_page+1):
              page = pdf_reader.pages[page_num].extract_text()
              print(page_num)

              # Remove figures, tables, author names, and titles

              page = re.sub(figure_regex, "", page, flags=re.IGNORECASE)

              page = re.sub(table_regex, "", page, flags=re.IGNORECASE)

              paragraphs = divide_into_paragraphs(page, num_qa_per_pg)
              for i, paragraph in enumerate(paragraphs):
                #print(i)

                if (count<num_qa):
                  print(count)
                  ques,answer = get_question1(paragraph,model1,tokenizer1)
                  print(ques)
                  #if(len(qa_dict) > 0):
                  if(process_question(ques,qa_dict.keys())==" "):
                    qa_dict[ques] = answer
                    count=count+1
                else:
                  break
  return qa_dict
my_dict={}
my_dict=qa_func(15,20,20)
print(my_dict)

Question_Similarity_from Vishnu

In [ ]:
x = my_dict.keys()
print(len(x))
for i in x:
  print(i)
#print(x)
y = my_dict.items()
print(y)#Full list
z = my_dict.values()
print(z)

In [ ]:
class STSBertModel(torch.nn.Module):

    def __init__(self):

        super(STSBertModel, self).__init__()

        word_embedding_model = models.Transformer('bert-base-uncased', max_seq_length=128)
        pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
        self.sts_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

    def forward(self, input_data):

        output = self.sts_model(input_data)

        return output

In [ ]:
!pip install sentence_transformers
import joblib
trained_model = joblib.load('/content/drive/MyDrive/Copy of Sbert_Model_1')

In [ ]:
!pip install datasets
!pip install sentence-transformers
!pip install transformers
!pip install torchmetrics.functional
import torch
from sentence_transformers import SentenceTransformer, models
from transformers import BertTokenizer
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm
from datasets import load_dataset
import matplotlib.pyplot as plt
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# Function to predict test data
def predict_sts(texts):

  trained_model .to('cpu')
  trained_model.eval()
  test_input = tokenizer(texts, padding='max_length', max_length = 128, truncation=True, return_tensors="pt")
  test_input['input_ids'] = test_input['input_ids']
  test_input['attention_mask'] = test_input['attention_mask']
  del test_input['token_type_ids']

  test_output = trained_model(test_input)['sentence_embedding']
  sim = torch.nn.functional.cosine_similarity(test_output[0], test_output[1], dim=0).item()

  return sim

In [ ]:
def process_question(question,questions):
  similarity = False
  compare_questions = []
  similar_question = " "
  if(len(question)>0):
    for value in questions:
      if (similarity == False):
        compare_questions.append(question)
        #print('Q',question)
        compare_questions.append(value)
        #print('V',value)
        if predict_sts(compare_questions) > 0.8 :
          similarity = True
          similar_question = value
        compare_questions.clear()
  return similar_question


In [ ]:
for i in my_dict.keys():
  a=process_question(i,my_dict.keys())
  print(a)

In [ ]:
for i in x:
  print(i)

In [ ]:
sentences1= [
        "The QuAC dataset promotes natural and diverse questions by collecting the data in an interactive setting with two crowd workers playing the roles of teacher and student. Unlike previous dialog-style QA datasets that use semi-automatic question generation, QuAC allows for genuine human interaction. This approach encourages a wider range of questions and avoids the limitations of automated question generation techniques.",
        "The QuAC dataset encourages the use of natural and diverse questions by gathering information in an engaging environment with two crowd workers acting as teacher and student. QuAC enables true human involvement in contrast to earlier dialog-style QA datasets that employ semi-automatic question generating. This strategy promotes a larger variety of queries while avoiding the drawbacks of automated question generation methods."
    ]

In [ ]:
predict_sts(sentences1)

In [ ]:
def process_answer(answer,answers):
  similar_answer = False
  compare_answers = []
  for value in answers:
    if (similarity == False):
      compare_answers.add(answer)
      compare_answers.add(value)
      if predict_sts(compare_answers) > 0.7:
        similar_answer = True
      compare_answers.clear()
  return similar_answer

In [ ]:
!pip install --quiet gradio==3.9

In [ ]:
import gradio as gr

context = gr.inputs.Textbox(lines=5,placeholder="Enter the page number upto which you want to extract question.")
# answer = gr.inputs.Textbox(lines=3, placeholder="Enter answer/keyword here...")
question = gr.outputs.Textbox( type="auto", label="Question")
#answer = gr.outputs.Textbox( type="auto", label="Answer")

def ques_answer(context):
  return get_question1(context,model1,tokenizer1)

iface = gr.Interface(
  fn=ques_answer,
  inputs=[context],
  outputs=question)


iface.launch(debug=False,share=True)

In [ ]:
out = []
threshold = int(len(text)/4)
#print(threshold)
for chunk in text.split('. '):
    if out and len(chunk)+len(out[-1]) < threshold:
        out[-1] += ' '+chunk+'.'
        print(out)
    else:
        #print(out)
        out.append(chunk+'.')
    print(out)

In [ ]:
text='According to one embodiment of the present dis closure , a question - answering learning method is provided . The question - answering learning method includes the following . A classifier module is created by a classifier gen eration module according to N1 labeled sentences among N sentences , wherein the classifier module includes several classifiers each representing different question - answering models , and both of the N and the N1 are positive integers . At least one corresponding sentence type of each of the N2 unlabeled sentences among the N sentences is determined by each of the classifiers , wherein the N2 is a positive integer . In a consistency evaluation process , N3 sentences are selected from the N2 unlabeled sentences by a consistency evaluation module according to a degree of consistency of determined results of the classifiers , wherein the determined results of the N3 sentences are determined by the classifiers and are inconsistent , and the N3 is a positive integer . In a complementarity evaluation process , N4 mutually complementary sentences are selected as to - be - labeled sentences from the N3 sentences by a complementarity evaluation module , wherein the N4 is a positive integer . After the N4 selected to - be - labeled sentences are labeled , the classifiers of the classifier module are re - created by the classifier generation module according to the N1 labeled sentences and the N4 selected to - be - labeled sentences . At least one of previously created classifiers is added to the classifier module to be members of the classifier module by a classifier evaluation module. According to another embodiment of the present disclosure , a question - answering learning system is provided . The question - answering learning system includes a classifier generation module , a consistency evaluation module , a complementarity evaluation module and a classifier evaluation module . The classifier generation module is configured to : create a classifier module according to N1 labeled sentences among N sentences , wherein the classifier module includes several classifiers each representing different ques tion - answering models , and both of the N and the N1 are positive integers . Each of the classifiers determines at least one corresponding sentence type of each of the N2 unlabeled sentences among the N sentences , wherein the N2 is a positive integer . The consistency evaluation module is configured to : select , in a consistency evaluation process , N3 sentences from the N2 unlabeled sentences according to a degree of consistency of determined results of the classifiers , wherein the determined results of the N3 sentences are determined by the classifiers and are inconsistent , and the N3 is a positive integer . The complementarity evaluation module is configured to : select , in a complementarity evalu ation process , N4 mutually complementary sentences as to - be - labeled sentences from the N3 sentences , wherein the N4 is a positive integer . The classifier generation module is further configured to : re - create , after the N4 selected to - be labeled sentences are labeled , the classifiers of the classifier module according to the N1 labeled sentences and the N4 selected to - be - labeled sentences . The classifier evaluation module is configured to : add at least one of previously created classifiers to the classifier module to be members of the classifier module According to an alternate embodiment of the pres ent disclosure , a computer programming product is disclosed . The computer program product is installed in a question - answering learning system to execute a question answering learning method . The question - answering learning method includes the following : A classifier module is created by a classifier generation module according to N1 labeled sentences among N sentences , wherein the classifier module includes several classifiers each representing different question - answering models , and both of the N and the N1 are positive integers . At least one corresponding sentence type of each of the N2 unlabeled sentences among the N sentences is determined by each of the classifiers , wherein the N2 is a positive integer . In a consistency evaluation process , N3 sentences are selected from the N2 unlabeled sentences by a consistency evaluation module according to a degree of consistency of determined results of the classifiers , wherein the determined results of the N3 sentences are determined by the classifiers and are inconsistent ,and the N3 is a positive integer . In a complementarity evaluation process , N4 mutually complementary sentences are selected as to - be - labeled sentences from the N3 sentences by a complementarity evaluation module , wherein the words or textual meanings of the N4 sentences are not mutually repetitive , similar , implicative or derivative , and the N4 is a positive integer . After the N4 selected to - be - labeled sentences are labeled , the classifiers of the classifier module are re - created by the classifier generation module according to the N1 labeled sentences and the N4 selected to - be - labeled sentences . At least one of previously created classifiers is added to the classifier module to be members of the classifier module by a classifier evaluation module The above and other aspects of the invention will become better understood with regards to the following detailed description of the preferred but non - limiting embodiment ( s ) . The following description is made with reference to the accompanying drawings '

In [ ]:
def ClusQuesGen(a,b,n):
#a=1st page of the book, b=last page of the pdf, n=number of questions you want from each page
      import PyPDF2
      from sentence_transformers import SentenceTransformer
      from sklearn.cluster import KMeans

      embedder = SentenceTransformer('all-MiniLM-L6-v2')
      reader = PyPDF2.PdfReader('/content/drive/MyDrive/2006.09595 (1).pdf')

      # Corpus with example sentences
      #corpus = ['A man is eating food. A man is eating a piece of bread. A man is eating pasta.The girl is carrying a baby. The baby is carried by the womanA man is riding a horse. A man is riding a white horse on an enclosed ground. A monkey is playing drumsSomeone in a gorilla costume is playing a set of drums. A cheetah is running behind its prey. A cheetah chases prey on across a field.']
      #for pg in range(0,len(reader.pages)):
      ques_answer=[]
      for pg in range(a,b):
        #print("New Para: "+ str(pg))
          context=reader.pages[pg].extract_text()
          corpus1 = context.split('. ')
          #print(corpus1)
          corpus_embeddings = embedder.encode(corpus1)
          num_clusters = n
          clustering_model = KMeans(n_clusters=num_clusters)
          clustering_model.fit(corpus_embeddings)
          cluster_assignment = clustering_model.labels_
          #print(cluster_assignment)
          clustered_sentences = [[] for i in range(num_clusters)]
          for sentence_id, cluster_id in enumerate(cluster_assignment):
            clustered_sentences[cluster_id].append(corpus1[sentence_id])
            #print(clustered_sentences)
          for i, cluster in enumerate(clustered_sentences):
            print("Context ", i+1)
            print(cluster)
            print("")
            ques, answer = get_question1(cluster,model1,tokenizer1)
            ques_answer.append([cluster, i+1, ques, answer])
            #return cluster, ques, answer
            #print(ques_answer)
            print ("question: ",ques)
            print ("answer: ",answer)
      #return cluster, ques, answer


In [ ]:
!pip install sentence_transformers
ClusQuesGen(2,3,1)

T5 Large on Race

In [ ]:
!pip install transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("potsawee/t5-large-generation-squad-QuestionAnswer")
model = AutoModelForSeq2SeqLM.from_pretrained("potsawee/t5-large-generation-squad-QuestionAnswer")


tokenizer1 = AutoTokenizer.from_pretrained("potsawee/t5-large-generation-race-QuestionAnswer")
model1 = AutoModelForSeq2SeqLM.from_pretrained("potsawee/t5-large-generation-race-QuestionAnswer")

context = r"""The Sino-Indian War also known as the China–India War, Indo-China War, Indo-China War of 1962 or Sino-Indian War of 1962, was a military conflict between China and India that took place from October to November 1962. It was a military escalation of the Sino-Indian border dispute. Fighting occurred along India's border with China, in India's North-East Frontier Agency east of Bhutan, and in Aksai Chin west of Nepal.""".replace('\n', ' ')
#context = a
for i in context:
      #print(context[i])

      inputs = tokenizer(context[i], return_tensors="pt",truncation=True,padding=True)
      outputs = model.generate(**inputs, max_length=100)
      question_answer = tokenizer.decode(outputs[0], skip_special_tokens=False)
      question_answer = question_answer.replace(tokenizer.pad_token, "").replace(tokenizer.eos_token, "")
      question, answer = question_answer.split(tokenizer.sep_token)
      print("extractive question:", question)
      print("extractive answer:", answer)

      inputs = tokenizer1(context[i], return_tensors="pt",truncation=True,padding=True)
      outputs = model1.generate(**inputs, max_length=100)
      question_answer = tokenizer1.decode(outputs[0], skip_special_tokens=False)
      question_answer = question_answer.replace(tokenizer1.pad_token, "").replace(tokenizer1.eos_token, "")
      question, answer = question_answer.split(tokenizer1.sep_token)
      print("abstractive question:", question)
      print("abstractive answer:", answer)

In [ ]:
t5('''The Sino-Indian War also known as the China–India War, Indo-China War, Indo-China War of 1962 or Sino-Indian War of 1962, was a military conflict between China and India that took place from October to November 1962. It was a military escalation of the Sino-Indian border dispute. Fighting occurred along India's border with China, in India's North-East Frontier Agency east of Bhutan, and in Aksai Chin west of Nepal.''')

T5 Base on Race

In [ ]:
import argparse
import glob
import os
import json
import time
import logging
import random
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
class QueGenerator():
  def __init__(self):
    self.que_model = T5ForConditionalGeneration.from_pretrained('./t5_que_gen_model/t5_base_que_gen/')
    self.ans_model = T5ForConditionalGeneration.from_pretrained('./t5_ans_gen_model/t5_base_ans_gen/')

    self.que_tokenizer = T5Tokenizer.from_pretrained('./t5_que_gen_model/t5_base_tok_que_gen/')
    self.ans_tokenizer = T5Tokenizer.from_pretrained('./t5_ans_gen_model/t5_base_tok_ans_gen/')

    self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    self.que_model = self.que_model.to(self.device)
    self.ans_model = self.ans_model.to(self.device)

  def generate(self, text):
        answers = self._get_answers(text)
        questions = self._get_questions(text, answers)
        output = [{'answer': ans, 'question': que} for ans, que in zip(answers, questions)]
        return output

  def _get_answers(self, text):
    # split into sentences
    sents = sent_tokenize(text)

    examples = []
    for i in range(len(sents)):
      input_ = ""
      for j, sent in enumerate(sents):
        if i == j:
            sent = "[HL] %s [HL]" % sent
        input_ = "%s %s" % (input_, sent)
        input_ = input_.strip()
      input_ = input_ + " </s>"
      examples.append(input_)

    batch = self.ans_tokenizer.batch_encode_plus(examples, max_length=512, pad_to_max_length=True, return_tensors="pt")
    with torch.no_grad():
      outs = self.ans_model.generate(input_ids=batch['input_ids'].to(self.device),
                                attention_mask=batch['attention_mask'].to(self.device),
                                max_length=32,
                                # do_sample=False,
                                # num_beams = 4,
                                )
    dec = [self.ans_tokenizer.decode(ids, skip_special_tokens=False) for ids in outs]
    answers = [item.split('[SEP]') for item in dec]
    answers = chain(*answers)
    answers = [ans.strip() for ans in answers if ans != ' ']
    return answers

  def _get_questions(self, text, answers):
    examples = []
    for ans in answers:
      input_text = "%s [SEP] %s </s>" % (ans, text)
      examples.append(input_text)

    batch = self.que_tokenizer.batch_encode_plus(examples, max_length=512, pad_to_max_length=True, return_tensors="pt")
    with torch.no_grad():
      outs = self.que_model.generate(input_ids=batch['input_ids'].to(self.device),
                                attention_mask=batch['attention_mask'].to(self.device),
                                max_length=32,
                                num_beams = 4)
    dec = [self.que_tokenizer.decode(ids, skip_special_tokens=False) for ids in outs]
    return dec

que_generator = QueGenerator()

text = "Python is an interpreted, high-level, general-purpose programming language. Created by Guido van Rossum \
  and first released in 1991, Python's design philosophy emphasizes code \
  readability with its notable use of significant whitespace."

text2 = "Gravity (from Latin gravitas, meaning 'weight'), or gravitation, is a natural phenomenon by which all \
  things with mass or energy—including planets, stars, galaxies, and even light—are brought toward (or gravitate toward) \
  one another. On Earth, gravity gives weight to physical objects, and the Moon's gravity causes the ocean tides. \
  The gravitational attraction of the original gaseous matter present in the Universe caused it to begin coalescing \
  and forming stars and caused the stars to group together into galaxies, so gravity is responsible for many of \
  the large-scale structures in the Universe. Gravity has an infinite range, although its effects become increasingly \
  weaker as objects get further away"
print(que_generator.generate(text2))

Bert

In [ ]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

#Model
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

#Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
question = '''What Nikhil finds?'''

paragraph = ''' 10 years earlier, Nikhil finds out that DJ had framed the evidence to send Shubh to prison, which is the reason why Nikhil resigned from the CBI. At present,
Nikhil is assigned DJ as the next victim and tells the killer that he has to go to jail to kill him since the central jail has high surveillance. The killer gets into the jail and as plotted,
 reaches DJ and is about to kill him'''

encoding = tokenizer.encode_plus(text=question, text_pair=paragraph)

inputs = encoding['input_ids']  # Token embeddings
sentence_embedding = encoding['token_type_ids']  # Segment embeddings
tokens = tokenizer.convert_ids_to_tokens(inputs)  # input tokens
start_scores, end_scores = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]),return_dict=False)
start_index = torch.argmax(start_scores)

end_index = torch.argmax(end_scores)

answer = ' '.join(tokens[start_index:end_index+1])
corrected_answer = ''

for word in answer.split():

    # If it's a subword token
    if word[0:2] == '##':
        corrected_answer += word[2:]
    else:
        corrected_answer += ' ' + word

print(question)
print(corrected_answer)

In [ ]:
from torchmetrics.functional import precision_recall
preds  = torch.tensor([2, 0, 2, 1])
target = torch.tensor([1, 1, 2, 0])
precision_recall(preds, target, average='macro', num_classes=3)

precision_recall(preds, target, average='micro')


In [ ]:
from torchmetrics import F1Score
target = torch.tensor([0, 1, 2, 0, 1, 2])
preds = torch.tensor([0, 2, 1, 0, 0, 1])
f1 = F1Score(num_classes=3)
f1(preds, target)

In [ ]:
!pip install anvil-uplink

In [ ]:
import anvil.server
anvil.server.connect("server_E3FCTYIOKPCC246OZC7ZI3IS-NOPF6GOREISCK3NF")
@anvil.server.callable
def value_check(file_data, N):
  print(file_data)
  print(N)
anvil.server.wait_forever()


In [ ]:
!pip install sent2vec

In [ ]:
pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("SparkByExamples.com") \
      .getOrCreate()


In [ ]:
dataList = [("Java", 20000), ("Python", 100000), ("Scala", 3000)]
rdd=spark.sparkContext.parallelize(dataList)
rdd2 = spark.sparkContext.textFile("/path/test.txt")

In [ ]:
data1 = [('James','','Smith','1991-04-01','M',3000),
  ('Michael','Rose','','2000-05-19','M',4000),
  ('Robert','','Williams','1978-09-05','M',4000),
  ('Maria','Anne','Jones','1967-12-01','F',4000),
  ('Jen','Mary','Brown','1980-02-17','F',-1)
]

columns = ["firstname","middlename","lastname","dob","gender","salary"]
df = spark.createDataFrame(data=data1, schema = columns)


In [ ]:
for i in df:
  print(i)

In [ ]:
df.show()
print(type(df))

In [ ]:
columns = ["firstname","middlename","lastname","dob","gender","salary"]
df = spark.createDataFrame(data=data, schema = columns)

In [ ]:
x=lambda a:a+a
print(x(4))